In [25]:
!nvidia-smi

Wed Jan 17 16:49:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [26]:
#!pip install simpletransformers

In [27]:
import json
with open(r"train.json", "r") as read_file:
  #Fatching the train data
    train = json.load(read_file)

In [28]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]}]

In [29]:
import json
with open(r"test.json", "r") as read_file:
  #Fatching the test data
    test = json.load(read_file)

In [30]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

In [31]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [40]:
model_type="bert"
model_name= "bert-base-cased"

if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [46]:
model_type="roberta"
model_name= "roberta-base"
# Configure the model
model_args = QuestionAnsweringArgs()  # Create an instance of the QuestionAnsweringArgs class

# Set the batch size for training
model_args.train_batch_size = 16

# Enable evaluation during training
model_args.evaluate_during_training = True

# Set the number of best predictions to consider during evaluation
model_args.n_best_size = 3

# Commenting out the following line to exclude the number of training epochs
model_args.num_train_epochs = 5


In [47]:
# Advanced Methodology Configuration
train_args = {
    "reprocess_input_data": True,  # Re-process input data during training
    "overwrite_output_dir": True,  # Overwrite the output directory if it already exists
    "use_cached_eval_features": True,  # Use cached evaluation features
    "output_dir": f"outputs/{model_type}",  # Set the output directory for the trained model
    "best_model_dir": f"outputs/{model_type}/best_model",  # Directory to save the best model
    "evaluate_during_training": True,  # Enable evaluation during training
    "max_seq_length": 128,  # Maximum sequence length for input data
    "num_train_epochs": 5,  # Number of training epochs
    "evaluate_during_training_steps": 1000,  # Evaluate during training every 1000 steps
    "wandb_project": "Question Answer Application",  # Weights and Biases project name
    "wandb_kwargs": {"name": model_name},  # Weights and Biases configuration for the run
    "save_model_every_epoch": False,  # Do not save the model after every epoch
    "save_eval_checkpoints": False,  # Do not save evaluation checkpoints
    "n_best_size": 3,  # Number of best predictions to consider
    # "use_early_stopping": True,  # Commented out: Use early stopping during training
    # "early_stopping_metric": "mcc",  # Commented out: Early stopping metric
    # "n_gpu": 2,  # Commented out: Number of GPUs to use
    # "manual_seed": 4,  # Commented out: Manual seed for reproducibility
    # "use_multiprocessing": False,  # Commented out: Use multiprocessing for data loading
    "train_batch_size": 128,  # Training batch size
    "eval_batch_size": 64,  # Evaluation batch size
    # "config": {
    #     "output_hidden_states": True
    # }  # Commented out: Model configuration option
}


In [48]:
model_name

'roberta-base'

In [49]:
model_type

'roberta'

In [50]:
model = QuestionAnsweringModel(model_type,model_name, args=train_args)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [51]:
model.train_model(train,eval_data=test)

add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 40233.13it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 0, 0, 0, 0],
  'similar': [2, 1, 1, 1, 1],
  'incorrect': [1, 2, 2, 2, 2],
  'train_loss': [4.8528642654418945,
   4.892578125,
   4.5185546875,
   4.455729007720947,
   4.2425127029418945],
  'eval_loss': [0.332763671875,
   0.09368896484375,
   0.09368896484375,
   -0.01357269287109375,
   -0.050140380859375]})

In [ ]:
### Remove output folder
!rm -rf outputs

In [52]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
result

{'correct': 0, 'similar': 1, 'incorrect': 2, 'eval_loss': -0.050140380859375}

In [54]:
texts

{'correct_text': {},
 'similar_text': {'00003': {'truth': '',
   'predicted': 'have all the',
   'question': 'What are Allomantic powers?'}},
 'incorrect_text': {'00001': {'truth': 'region called the Final Empire',
   'predicted': 'are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
   'question': 'Where does the series take place?'},
  '00002': {'truth': 'one',
   'predicted': 'have all the',
   'question': 'How many powers does a Misting possess?'}}}

In [57]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

In [58]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4198.50it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['great power and skill.']}]


In [59]:
probabilities

[{'id': '0', 'probability': [0.545724553814415]}]